In [1]:
export ReservoirShuffler

"""Shuffling an infinite list in finite memory.
Not perfectly random -- items are likely to remain close to each other, particularly at the start.
The larger the reservoir_size, the better the shuffle is
"""
immutable ReservoirShuffler{B}
    source::B
    reservoir_size::Int
end


function Base.start(it::ReservoirShuffler)
    reservoir, backer = Base.head_and_tail(it.source, it.reservoir_size)
    (reservoir, backer, Base.start(backer))
end

function Base.done(it::ReservoirShuffler, state)
    reservoir, backer, backer_state = state
    Base.done(backer, backer_state) && length(reservoir)==0
end

function Base.next(it::ReservoirShuffler, state)
    reservoir, backer, backer_state = state
    
    ret_index = rand(1:length(reservoir))
    ret = reservoir[ret_index]
    if Base.done(backer, backer_state) 
        splice!(reservoir, ret_index)
        # Nothing to replace it with
    else
        replacement, backer_state = Base.next(backer, backer_state)
        reservoir[ret_index] = replacement
    end
    ret, (reservoir, backer, backer_state)
end

Base.iteratorsize{B}(::Type{ReservoirShuffler{B}}) = Base.iteratorsize(B)
Base.eltype{B}(::Type{ReservoirShuffler{B}}) = Base.eltype(B)
Base.length(it::ReservoirShuffler) = length(it.source)
Base.size(it::ReservoirShuffler) = size(it.source)

    

In [22]:
using Base.Test
using Iterators

@testset "Reservoir  Shuffle" begin
	srand(15)
	
    # Make sure we are not losing anything
	@test ReservoirShuffler(1:10, 2) |> Set == 1:10 |> Set
	@test ReservoirShuffler(1:100, 2) |> Set == 1:100 |> Set
    @test ReservoirShuffler(1:1_000_000, 1024) |> Set == Set(1:1_000_000)
	@test ReservoirShuffler(1:1_000_000, 64) |> Set == Set(1:1_000_000)
    @test ReservoirShuffler(10:10:100, 64) |> Set == 10:10:100 |> Set
	

	@test ReservoirShuffler(1:1000, 1) |> collect == collect(1:1000) # with reservoir of 1 it is exactly the same
	@test ReservoirShuffler(1:1000, 10) |>collect != collect(1:1000) #Not same order
	@test ReservoirShuffler(1:1000, 1024) |> collect != collect(1:1000) #not same order

    #Try something infinite
    xs = take(ReservoirShuffler(iterate(x->x-0.5x, big"1.0"), 100), 1000) |> collect
    @test all(0 .<= xs)
    @test all(xs .<= 1)
    @test !issorted(xs)
    @test !issorted(reverse(xs))
    
end;

Test Summary:      | Pass  Total
  Reservoir  Shuffle |   12     12


In [19]:
 3 .<= 1

false

In [24]:
using Iterators

natural_numbers() = iterate(x->x+1 , 0)

natural_numbers (generic function with 1 method)

In [61]:
using Plots
gr()

Plots.GRBackend()

In [87]:
function plot_shift{T}(len::T, res_size, rounds)
    point_shifts = Matrix{T}(len, rounds)
    for round in 1:rounds
        for (ii, vv) in enumerate(ReservoirShuffler(1:len, res_size))
            point_shifts[ii, round] =  abs(ii - vv)
        end
    end
    
    tt = repeat(1:len, outer=rounds)
    scatter(tt, vec(point_shifts))
    plot!(1:len, vec(mean(point_shifts,2)))
end



plot_shift (generic function with 1 method)

In [ ]:
ani = @animate for rr in 50:50:1000
    plot_shift(1000, rr, 30)
    title!(string(rr))
end
gif(ani, fps=5)